In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
import scipy.linalg as la

from scipy.linalg import expm
from qiskit.quantum_info import Statevector, partial_trace, Operator, DensityMatrix
from qiskit.visualization import plot_state_qsphere, plot_histogram, plot_bloch_multivector

np.set_printoptions(precision=3)

In [12]:
def get_H0(N):
    """The uniform matrix"""
    v = np.ones((N,1))*(1/np.sqrt(N))
    return np.matmul(v, v.T.conj())

def get_H0_r(N):
    m = np.matrix([[1/N, np.sqrt(N-1)/N],
                  [np.sqrt(N-1)/N, (N-1)/N]], dtype=complex)
    return m

def get_v0(N):
    return np.ones((N,1))*(1/np.sqrt(N))
    
def get_vp(N):
    pv = np.zeros((N,1))
    pv[N-1] = 1
    return pv
    
def get_Hp(N):
    pv = problem_vector(N)
    return np.matmul(pv, pv.T.conj())

def get_Hp_r():
    m = np.matrix([[1, 0],
                  [0, 0]])
    return m

def get_H_op(delt, n, H):
    """
    Input:
        delt: time interval
        n: number of qubits
        H: Hamiltonian matrix
    return Operator exp(-itH)
    """
    H = H*(-1j*delt)
    exp_H = expm(H)
    h = QuantumCircuit(n, name = 'H_op')
    h.unitary(exp_H, range(n))
    return h.to_gate()

def get_f(t, epsilon, N):
    """
    Time evolve function
    Input:
        t: time point
        epsilon: error value
        N: 2**n
    return value from 0 to 1
    """
    arg = ((2*epsilon*np.sqrt(N-1)*t)/N - np.arctan(np.sqrt(N-1)))
    num = np.tan(arg) / np.sqrt(N-1)
    den = 1 + num
    result = 0.5 * den
    return result

def get_adiabatic_T(epsilon, N):
    t1 = 1/(2*epsilon)
    t2 = N/np.sqrt(N-1)
    t3 = np.arctan((np.sqrt(N-1)))+np.arctan(np.sqrt(N-1))
    return t1*t2*t3

def get_simulation_T(a):
    return np.pi/(2*a)

def get_R(epsilon, N):
    """get the total iteration R"""
    return (int)(np.sqrt(N)/(epsilon**3))

def get_state(qc):
    m = DensityMatrix(qc)
    v = m.to_statevector()
    return v
    
def get_Epsilon1(final_vector, m):
    """
    get the epsilon of the result, should lower than the theoretical epsilon

    """
    distance_vector = final_vector.copy()
    distance_vector[m-1] -= 1
    values = np.matmul(distance_vector.conj().T,distance_vector)
    return np.sqrt(values)

def get_Epsilon2(v1, v2):
    """return from 0 to 1"""
    res = np.matmul(v1.T.conj(), v2)
    return (1-(abs(res))**2)[0]

def get_eH(t, H):
    m = H*(-1j*t)
    exp_H = expm(m)
    return exp_H

def get_norm(v):
    return np.sqrt(np.matmul(v.T.conj(), v))

def normalize_vector(v):
    n = get_norm(v)
    v = v/n
    return v

In [65]:
n=4
N=2**n

a = 1/np.sqrt(N)
T = get_simulation_T(a)

state = np.zeros((2,1))
state[0] = 1
H0 = get_H0_r(N)
Hp = get_Hp_r()
eH = get_eH(T, H0+Hp)
new_state = np.dot(eH, state)

prim_state = np.matrix([[1/np.sqrt(N)],
                        [np.sqrt(N-1)/np.sqrt(N)]])

print(new_state)
print(prim_state)
# plot_state_qsphere(Statevector(new_state))

[[-9.958e-17-0.25j ]
 [ 2.372e-16-0.968j]]
[[0.25 ]
 [0.968]]


In [79]:
n=2
N=2**n

a = 1/np.sqrt(N)
T = get_simulation_T(a)/4

state = np.zeros((2,1))
state[0] = 1
H0 = get_H0_r(N)
Hp = get_Hp_r()
eH0 = get_eH(T, H0)
eHp = get_eH(T, Hp)
eH = np.dot(eH0, eHp)
new_state = np.dot(eH, state)
print(new_state)

prim_state = np.matrix([[1+(np.exp(-1j*T)-1)/N],
                        [np.sqrt(N-1)*(np.exp(-1j*T)-1)/N]])
print(prim_state)

[[ 0.53 -0.78j ]
 [-0.306-0.127j]]
[[ 0.927-0.177j]
 [-0.127-0.306j]]
